In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from scipy import sparse
#import time

#import matplotlib.pyplot as plt

#from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch.optim.swa_utils import AveragedModel
#from tqdm import tqdm

from torch_lr_finder import LRFinder

import utils

from importlib import reload
reload(utils)

<module 'utils' from '/home/artemy/multimodal_proj/nb/utils.py'>

In [2]:
cuda_n = 0
device = torch.device(f"cuda:{cuda_n}")

In [9]:
class Universal_Model(nn.Module, utils.HyperParameters):
    def __init__(self,
                 input_dim: int,
                 output_dim: int,
                 layers_sizes = [512] * 4,
                 concat_pos = 4,
                 n_of_layers_to_concat = 3,
                 dropout = 1,
                 device = 'cpu',
                 **kwargs 
    ):
        super().__init__()
        self.save_hyperparameters()
        self.layers_sizes = [self.input_dim] + self.layers_sizes + [self.output_dim]
        self.n_layers = len(self.layers_sizes) - 1
        modules = []
        for i in range(self.n_layers):
            if i != self.concat_pos:
                input_dim = self.layers_sizes[i]
            else:
                input_dim = sum(self.layers_sizes[self.concat_pos-self.n_of_layers_to_concat+1:self.concat_pos+1])
            output_dim = self.layers_sizes[i + 1]
            if i < self.n_layers and dropout != 1:
                modules.append(nn.Dropout1d(dropout))
            modules.append(nn.Linear(input_dim, output_dim))
            if i < self.n_layers - 1:
                modules.append(nn.BatchNorm1d(num_features=output_dim))          
                modules.append(nn.SiLU())
        self.net = nn.Sequential(*modules)
        self.net.apply(utils.init_weights)
    
    def forward(self, x):
        fc_layer_n = 0
        layer_outputs = []
        for module in self.net:
            if fc_layer_n == self.concat_pos and isinstance(module, nn.Linear):
                x = torch.concat(layer_outputs[-self.n_of_layers_to_concat:], 1)
            x = module(x)
            if isinstance(module, nn.SiLU):
                layer_outputs.append(x)
            if isinstance(module, nn.Linear):
                fc_layer_n += 1
        return x

In [15]:
def load_data(data_dir, train=True, cite=True):
    inputs_file = data_dir
    targets_file = data_dir

    if cite: 
        inputs_file += 'cite_gex_'
        targets_file += 'cite_adt_'
    else:
        inputs_file += 'atac_'
        targets_file += 'gex_'
    if train:
        inputs_file += 'train'
        targets_file += 'train'
    else:
        inputs_file += 'test'
        targets_file += 'test'
        
    data = {}
    inputs = utils.load_sparse_data(inputs_file + '_svd.sparse.npz')
    data['inputs'] = inputs
    targets = utils.load_sparse_data(targets_file + '.sparse.npz')
    data['targets'] = targets
        
    if cite:
        cycle_levels = utils.load_sparse_data(inputs_file + '_cycle.sparse.npz')
        data['cycle'] = cycle_levels
        cd_levels = utils.load_sparse_data(inputs_file + '_cd.sparse.npz')
        data['cd'] = cd_levels
        cycle_levels_imputed = utils.load_sparse_data(inputs_file + '_imputed_cycle.sparse.npz')
        data['cycle_imputed'] = cycle_levels_imputed
        cd_levels_imputed = utils.load_sparse_data(inputs_file + '_imputed_cd.sparse.npz')
        data['cd_imputed'] = cd_levels_imputed
    
    return data
    

In [18]:
def configure_data_loaders(data,
                           n_of_PCs=48,
                           cite=True,
                           cd=False,
                           cycle=False,
                           imputed=False,
                           batch_size=2048,
                           train=True):
    
    selected_data = []
    selected_data.append(data['inputs'][:, :n_of_PCs])
    if imputed:
        if cd:
            selected_data.append(data['cd_imputed'])
        if cycle:
            selected_data.append(data['cycle_imputed'])
    else:
        if cd:
            selected_data.append(data['cd'])
        if cycle:
            selected_data.append(data['cycle'])

    inputs = np.concatenate(selected_data, 1)
    targets = data['targets']
    
    if train:
        train_loader, val_loader = utils.make_loaders(inputs, targets, val_size=2048 * 2, batch_size=batch_size, num_workers=1)
        return train_loader, val_loader
    else:
        test_loader = utils.make_loaders(inputs, batch_size=batch_size, num_workers=1)
        return test_loader

In [19]:
def run_model(model, model_params, trainer_params, train_loader, val_loader):
    torch.cuda.empty_cache()
    trainer = utils.Trainer(**trainer_params)
    trainer.add_train_loader(train_loader)
    trainer.add_val_loader(val_loader)
    trainer.add_model(model, model_params)
    val_loss = trainer.fit(return_val_loss=True, verbose=False)
    return val_loss

In [20]:
import optuna


def objective(trial, data, model, device, cite=True):
    
    model_params = {}
    num_layers = trial.suggest_int("num_layers", 1, 25)
    model_params['layers_sizes'] = []
    for layer in range(num_layers):
        ls = trial.suggest_int(f"ls_{layer}", 10, 10000)
        model_params['layers_sizes'].append(ls)
    
    model_params['concat_pos'] = trial.suggest_int("concat_pos", 1, num_layers)
    max_concat_pos = max(1, model_params['concat_pos']-1)
    model_params['n_of_layers_to_concat'] = trial.suggest_int("n_of_layers_to_concat", 1, max_concat_pos)
    
    model_params['dropout'] = trial.suggest_float("dropout", 0, 1)
    
    
    trainer_params = {}
    trainer_params['device'] = device
    
    trainer_params["wd"] = trial.suggest_float("wd", 1e-7, 1e1, log=True)
    trainer_params["lr"] = trial.suggest_float("lr", 1e-5, 1e1, log=True)
    

    
    
    trainer_params['max_epochs'] = trial.suggest_int("max_epochs", 1, 50)
    trainer_params['max_schedule_epoch'] = trial.suggest_int("max_schedule_epoch", 1, trainer_params['max_epochs'])
    if trainer_params['max_epochs'] != trainer_params['max_schedule_epoch']:
        trainer_params['min_lr'] = trial.suggest_float("min_lr", 1e-5, 1e-1, log=True)
    
    trainer_params['sparsity_beta'] = trial.suggest_float("sparsity_beta", 1e-9, 10, log=True)
    trainer_params['sparsity_rho'] = trial.suggest_float('sparsity_rho', 1e-5, 0.05, log=True)

    # regularization
    trainer_params['l1_weight'] = trial.suggest_float('l1_weight', 1e-6, 1, log=True)
    trainer_params['l2_weight'] = trial.suggest_float('l2_weight', 1e-6, 1, log=True)
    
    trainer_params['use_swa'] = trial.suggest_categorical('use_swa', [True, False])
    if trainer_params['use_swa']:
        trainer_params['swa_start'] = trial.suggest_int("swa_start", 1, trainer_params['max_epochs'])
        trainer_params["swa_lr"] = trial.suggest_float("swa_lr", 1e-5, 1e1, log=True)
    
    if not trainer_params['use_swa']:
        trainer_params['use_one_cycle'] = trial.suggest_categorical('use_one_cycle', [True, False])
        
    
    n_of_PCs = trial.suggest_int("n_of_PCs", 5, 512)
    if cite:
        cd = trial.suggest_categorical('cd', [True, False])
        cycle = trial.suggest_categorical('cycle', [True, False])
        imputed = trial.suggest_categorical('imputed', [True, False])
    
    batch_size = trial.suggest_categorical("batch_size", [512, 1024, 2048, 4096])
    
    train_loader, val_loader = configure_data_loaders(data,
                                                       n_of_PCs=n_of_PCs,
                                                       cite=cite,
                                                       cd=cd,
                                                       cycle=cycle,
                                                       imputed=imputed,
                                                       batch_size=batch_size)
    
    val_loss = run_model(model, model_params, trainer_params, train_loader, val_loader)
    
    return -val_loss[1]

In [21]:
data_dir = '/home/artemy/multimodal_proj/data/tuning_data/'

data = load_data(data_dir, train=True, cite=True)

In [22]:


study = optuna.create_study()
initialized_objective = lambda x: objective(x, model=Universal_Model, data=data, device=device, cite=True)
study.optimize(initialized_objective, n_trials=1000)





[I 2022-10-26 01:07:55,006] A new study created in memory with name: no-name-4dc25676-b0f4-4aea-bf57-acec081ed3cc
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:09<00:00,  1.24s/it]
[I 2022-10-26 01:08:07,310] Trial 0 finished with value: -0.7495240569114685 and parameters: {'num_layers': 5, 'ls_0': 4700, 'ls_1': 346, 'ls_2': 7351, 'ls_3': 490, 'ls_4': 2659, 'concat_pos': 2, 'n_of_layers_to_concat': 1, 'dropout': 0.8167667578685678, 'wd': 1.8049363774415097, 'lr': 0.10436849345463711, 'max_epochs': 8, 'max_schedule_epoch': 8, 'sparsity_beta': 0.0010258683293824704, 'sparsity_rho': 0.0300673129187057, 'l1_weight': 1.7333063763915774e-06, 'l2_weight': 0.007351678753967521, 'use_swa': False, 'use_one_cycle': False, 'n_of_PCs': 97, 'cd': True, 'cycle': False, 'imputed': True, 'batch_size': 2048}. Best is trial 0 with value: -0.7495240569114685.
100%|█████████████████████████████████████████████████████████████████████████

RuntimeError: CUDA out of memory. Tried to allocate 210.00 MiB (GPU 0; 23.69 GiB total capacity; 21.59 GiB already allocated; 14.56 MiB free; 22.74 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF